# Композиции алгоритмов классификации

## Необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

## Подготовка данных

In [2]:
path_to_file = '../datasets/classification/occupancy_detection_preprocessed.csv'
dataset = pd.read_csv(path_to_file)

In [3]:
dataset.head()

,Unnamed: 0,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,140,2015-02-02 14:19:00,23.7000,26.272,585.200000,749.200000,0.004764,1
1,141,2015-02-02 14:19:59,23.7180,26.290,578.400000,760.400000,0.004773,1
2,142,2015-02-02 14:21:00,23.7300,26.230,572.666667,769.666667,0.004765,1
3,143,2015-02-02 14:22:00,23.7225,26.125,493.750000,774.750000,0.004744,1
4,144,2015-02-02 14:23:00,23.7540,26.200,488.600000,779.000000,0.004767,1


Удалим ненужные столбцы, разобьём выборку и отмасштабируем

In [5]:
dataset = dataset.drop(['Unnamed: 0', 'date'], axis=1)

In [6]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 5].values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [8]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Бэггинг

**Бэггинг** – способ постороения композиции алгоритмов, при котором каждый базовый аллгоритм обучается на подвыборке исходной обучающей выборки. Формирование подвыборке происходит при помощи *бутстрапа* (выбор с возвращением элементов из исходного множества)

## Стэкинг

**Стэкинг**: на основе обученных базовых алгоритмов строится новый *метаалгоритм*, для вывода предсказаний, основанных на множественных предсказаниях, возвращаемых этими базовыми алгоритмами

## Градиентный бустинг

**Градиентный бустинг**: градиентный бустинг сводит задачу к градиентному спуску: на каждой итерации мы подгоняем слабого ученика к антиградиенту текущей ошибки подбора по отношению к текущей модели ансамбля